In [1]:
import cv2
import glob
import numpy as np

import matplotlib.pyplot as plt
from progressbar import *

from MatlabTests.python.read_write_model import *
from MatlabTests.python.read_dense import *

In [2]:
def rank_images(images):
    ids = []
    for image in images:
        id_str = image.split('\\')[-1][:-4]
        ids.append(int(id_str))
        
    rank = np.argsort(np.asarray(ids))
    return [images[i] for i in rank]

In [3]:
def resize_data(data, size = (480,640)):
    #result = np.zeros(data.shape)
    if(len(data.shape) == 2):
        # depth
        result = np.zeros((size[0], size[1]))
        result[0:data.shape[0], 0:data.shape[1]] = data
    else:
        result = np.zeros((size[0], size[1], 3)).astype(np.uint8)
        result[0:data.shape[0], 0:data.shape[1], :] = data
    return result

In [4]:
def combine_two_image(img1, img2, margin=10):
    H0, W0, _ = img1.shape
    H1, W1, _ = img2.shape
    H, W = max(H0, H1), W0 + W1 + margin
    out_all = 255*np.ones((H, W, 3), np.uint8)
    out_all[:H0, :W0, :] = img1
    out_all[:H1, W0+margin:, :] = img2
    return out_all

def combine_two_image_vertical(img1, img2, margin=10):
    if(img1 is None):
        return img2
    H0, W0, _ = img1.shape
    H1, W1, _ = img2.shape
    H, W = H0 + H1 + margin, max(W0, W1)
    out_all = 255*np.ones((H, W, 3), np.uint8)
    out_all[:H0, :W0, :] = img1
    out_all[H0+margin:, :W1, :] = img2
    return out_all

def put_topleft_text(image, text):
    sc = min(image.shape[0] / 640., 2.0)
    Ht = int(30 * sc)
    cv2.putText(image, text, (int(8*sc), Ht), cv2.FONT_HERSHEY_DUPLEX,
                    1.0*sc, (0, 255, 0), 2, cv2.LINE_AA)
    return image

In [20]:
worksapce_path = "./WinterThirdFloor/winter_third_floor_results/"
images_folder = worksapce_path + "/images/"
sub_folders = ['cut_'+str(i+1) for i in range(3)]
images_all = []
for sub_folder in sub_folders:
    images = glob.glob(images_folder+ sub_folder + "/*.png")
    images = rank_images(images)
    images_all = images_all + images
print("==> Loaded", len(images_all), "images")

==> Loaded 4419 images


# 1. Make Segmentation Videos

In [28]:
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
height = 480
width = 640
margin = 10
video_writer = cv2.VideoWriter("WinterThirdFloor/output.avi", fourcc, 20, (width*3+margin*2, height))

bar = ProgressBar()
for i in bar(range(len(images_all))):
    image_name = images_all[i].split("/")[-1]
        
    rgb = cv2.imread(images_all[i])
    rgb = resize_data(rgb, (height,width))
    rgb = put_topleft_text(rgb, "rgb image "+ image_name + " " + str(i))
    
    seg_ade20k = cv2.imread(worksapce_path+"/segmentation/"+image_name+".png")
    seg_ade20k = cv2.resize(seg_ade20k, (width, height))
    seg_ade20k = put_topleft_text(seg_ade20k, "ade20k trained")
    
#     image_name = images[i].split("\\")[-1]
#     seg_pascal = cv2.imread(worksapce_path+"/segmentation_pascal/"+image_name+".png")
#     seg_pascal = cv2.resize(seg_pascal, (width, height))
#     seg_pascal = put_topleft_text(seg_pascal, "pascal trained")
    
    depth_path = worksapce_path + '/depth/' + image_name + '.geometric.bin'
    mvs_depth = read_array(depth_path)
    mvs_depth = cv2.applyColorMap(cv2.convertScaleAbs(mvs_depth,alpha=30),cv2.COLORMAP_JET)
    mvs_depth = resize_data(mvs_depth, (height,width))
    mvs_depth = put_topleft_text(mvs_depth, "colmap mvs")
    
    frame = combine_two_image(rgb, seg_ade20k, margin)
    frame = combine_two_image(frame, mvs_depth, margin)
    video_writer.write(frame)
    
video_writer.release()

100% |########################################################################|


# 2. Make depth prediction video

In [11]:
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
height = 480
width = 640
margin = 10
video_writer = cv2.VideoWriter("garden/output_depth.avi", fourcc, 20, (width*3+margin*2, height*2+margin))

bar = ProgressBar()
for i in bar(range(len(images))):
    rgb = cv2.imread(images[i])
    rgb = resize_data(rgb, (height,width))
    rgb = put_topleft_text(rgb, "rgb image")
    
    image_name = images[i].split("\\")[-1][:-4]
    
    demon = cv2.imread(worksapce_path+"/depth_completion/demon/" +image_name+"_depth.png")
    demon = cv2.resize(demon, (width, height))
    demon = put_topleft_text(demon, "demon")
    
    deepmvs = np.load(worksapce_path+"/deepmvs/" +image_name+".png.output.npy")
    deepmvs = cv2.applyColorMap(cv2.convertScaleAbs(deepmvs,alpha=30),cv2.COLORMAP_JET)
    deepmvs = cv2.resize(deepmvs, (width, height))
    deepmvs = put_topleft_text(deepmvs, "deepmvs")
    
    full_depth = combine_two_image(rgb, demon, margin)
    full_depth = combine_two_image(full_depth, deepmvs, margin)
    
#     depth_path = worksapce_path + '/mvs/stereo/depth_maps/' + image_name + '.png.geometric.bin'
#     mvs_depth = read_array(depth_path)
#     mvs_depth = cv2.applyColorMap(cv2.convertScaleAbs(mvs_depth,alpha=30),cv2.COLORMAP_JET)
#     mvs_depth = resize_data(mvs_depth, (height,width))
#     mvs_depth = put_topleft_text(mvs_depth, "colmap mvs")
    
    sparse = cv2.imread(worksapce_path+"/depth_completion/sparse/"+image_name+"_dep.jpg")
    sparse = cv2.resize(sparse, (width, height))
    sparse = put_topleft_text(sparse, "sparse features")
    
#     colmap = combine_two_image(rgb, mvs_depth, margin)
#     colmap = combine_two_image(colmap, sparse, margin)
    
    cspn = cv2.imread(worksapce_path+"/depth_completion/cspn/"+image_name+"_dep.jpg")
    cspn = cv2.resize(cspn, (width, height))
    cspn = put_topleft_text(cspn, "cspn")
    
    sparse_to_dense = cv2.imread(worksapce_path+"/depth_completion/sparse_to_dense/"+image_name+"_dep.jpg")
    sparse_to_dense = cv2.resize(sparse_to_dense, (width, height))
    sparse_to_dense = put_topleft_text(sparse_to_dense, "sparse_to_dense")
    
    from_sparse = combine_two_image(sparse, cspn, margin)
    from_sparse = combine_two_image(from_sparse, sparse_to_dense, margin)

    frame = combine_two_image_vertical(full_depth, from_sparse, margin)
    video_writer.write(frame)
    
video_writer.release()

100% |########################################################################|


# 3. Make Floor Repair video

In [10]:
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
height = 480
width = 640
margin = 10
video_writer = cv2.VideoWriter("garden/output_floor.avi", fourcc, 20, (width*3+margin*2, height*2+margin))

bar = ProgressBar()
for i in bar(range(len(images))):
    rgb = cv2.imread(images[i])
    rgb = resize_data(rgb, (height,width))
    rgb = put_topleft_text(rgb, "rgb image")
    
    image_name = images[i].split("\\")[-1][:-4]
    depth_path = worksapce_path + '/mvs/stereo/depth_maps/' + image_name + '.png.geometric.bin'
    mvs_depth = read_array(depth_path)
    mvs_depth = cv2.applyColorMap(cv2.convertScaleAbs(mvs_depth,alpha=30),cv2.COLORMAP_JET)
    mvs_depth = resize_data(mvs_depth, (height,width))
    mvs_depth = put_topleft_text(mvs_depth, "colmap mvs")
    
    seg_ade20k = cv2.imread(worksapce_path+"/segmentation_ade20k/"+image_name+".png.png")
    seg_ade20k = cv2.resize(seg_ade20k, (width, height))
    seg_ade20k = put_topleft_text(seg_ade20k, "ade20k trained")
    
    source = combine_two_image(rgb, mvs_depth, margin)
    source = combine_two_image(source, seg_ade20k, margin)
    
    floor_repair = cv2.imread(worksapce_path+"/floor_process/del_floor/"+image_name+".png.jpg")
    floor_repair = resize_data(floor_repair, (height,width))
    floor_repair = put_topleft_text(floor_repair, "floor repair")
    
    floor_fill = cv2.imread(worksapce_path+"/floor_process/fill_floor/"+image_name+".png.jpg")
    floor_fill = resize_data(floor_fill, (height,width))
    floor_fill = put_topleft_text(floor_fill, "floor filling")

    depth_complete = np.load(worksapce_path+"/tv_res/matlab/"+image_name+".png.npy")
    depth_complete = cv2.applyColorMap(cv2.convertScaleAbs(depth_complete,alpha=30),cv2.COLORMAP_JET)
    depth_complete = resize_data(depth_complete, (height,width))
    depth_complete = put_topleft_text(depth_complete, "TV reconstruction")
    
    result = combine_two_image(floor_repair, floor_fill, margin)
    result = combine_two_image(result, depth_complete, margin)

    frame = combine_two_image_vertical(source, result, margin)
    video_writer.write(frame)
    
video_writer.release()

100% |########################################################################|
